<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#처리-단계-명세" data-toc-modified-id="처리-단계-명세-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>처리 단계 명세</a></span></li></ul></div>

In [2]:
import modules.newsight as ns 

# file I/O
import pickle
import pandas as pd

import time # check excution time

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings("ignore")

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#처리-단계-명세" data-toc-modified-id="처리-단계-명세-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>처리 단계 명세</a></span></li></ul></div>

## 처리 단계 명세

**DB** <font color='red'>workflow v3 전처리(토큰화).ipynb</font>  

1. **검색어 입력**
2. **데이터 불러오기**(raw data)   
1. **전처리**  
    1.1. 특수문자, 일반적 불용어 제거  
    1.2. 유효 명사 추가     
    1.4. 검색어에 대한 불용어 제거  
    1.3. 명사 추출  

**Backend** workflow v3 유효기사추출 - 대표기사추출.ipynb 

1. **검색어 입력**
2. **데이터 불러오기**(tokenized data)
3. **유효문서 추출**  
    3.2. 유효 문서 추출 A (제목에 검색어를 포함하는 문서만 추출, 제목에 [ ]가 들어간 문서 제외)  
    3.3. 유효 문서 추출 B (본문에 검색어가 3번 이상 들어가는 문서만 추출) 
4. **문서별 카테고리 할당**
5. **카테고리 별 토픽 할당**  
    5.1. 벡터화 (TFIDF)  
    5.2. 군집화 (DBSCAN)
6. **토픽 별 대표기사 추출**
    6.1. 토픽별 문서 벡터화 (TFIDF)  
    6.2. 토픽별 대표 키워드 추출 (SVD)  
    6.3. 토픽별 대표 문서 추출 (BM25)  
    
**Frontend**  workflow v3 시각화.ipynb 

    
**References**  
[1] 현호\_mulcam_mirae_workflow.ipynb

**기타**  
각 블록 마다 running time 측정

In [3]:
loader = ns.Pickle2DF()
df_raw = loader.get_dataframe('data/미래에셋대우/rawdata_미래에셋대우.pickle')

In [4]:
tic = time.time()

# 전처리 
pp = ns.PreprocessingText()
content_no_stopwords = pp.make_content_re(df_raw['content']) # 일반적인 불용어, 특수문자 제거

# 유효 명사 추가
noun_dict_route = 'data/미래에셋_단어사전.txt'
with open(noun_dict_route,encoding = 'utf-8-sig') as f: noun_proper = f.read()
noun_list = noun_proper.split('\n')
pp.add_noun_dict(noun_list) 

# 검색어에 대한 불용어 추가
stopword_dict_route = 'data/미래에셋_불용어사전.txt'
with open(stopword_dict_route,encoding = 'utf-8-sig') as f2: stopword = f2.read()
stopword_list = stopword.split('\n')
pp.add_stopwords(stopword_list) 

# 명사 추출
content_tk = pp.tokenize(content_no_stopwords)

toc = time.time()
print(f'excution time : {toc-tic}')

추가한 명사
['정보통신', '환헤지', '조웅기', '핀테크', '큐레이션', '투명성', '최경주', '상장', '임직원', '아시아그로쓰', '라이징스타', '파이낸셜', '캡스', '리파이낸싱', '싱가포르투자청', '도이체방크', '도이치뱅크', '카우룽', '그린마크', '광주은행', '대신증권', '삼성생명', '신한은행', '우리은행', '유안타증권', '펀드슈퍼마켓', '한국투자증권', '기업은행', '하나은행', '제일은행', '퀀트모델', '포트폴리오', '헤지펀드', '트레이딩', '영업점', '셀렉션', '구인', '사업화', '협약식', '미래에셋센터원', '최연소', '이테크', '미래에셋', '미래에셋대우', '에트리홀딩스', '펀드설정액', '분기', '벤처', '사회적기업', '성장지원', '헬스케어', '해결책', '메리츠', '신고가', '지주', '증권주', '벨류에이션', '이베스트', '수익성', '실사구시', '페어트레이딩', '쓸어', '상장사', '한투', '한국투자증권', '순매수', '거래량', '상장주식수', '분기', '미상환', '업무지구', '장기적', '한온시스템', '보여', '주간사', '젠바디', '예탁결제원', '크라우드펀딩', '영업이익률', '이익률', '파워리퍼블릭', '운용사', '아모텍', '지분', '박스권', '저평가', '시총', '긍정적', '납세자', '셀리드', '에스퓨얼셀', '게임사', '할인율', '재도전', '인공지능', '당기순익', '비대면', '사업가치', '재평가', '순매도', '순매수', '수혜주', '마중물', '혼조세', '신고가', '합리적', '재취업', '다각화', '자문형랩', '순익', '설정액', '원소재', '메디톡스', '인허가', '성장통', '호실적', '주간사', '주관사', '예상치', '호실적', '압도', '신성장', '기술료', '조기상환', '비임원', '기대감', '순매도', '저평가', '거래량', '내수', '건설사', 

In [5]:
tic = time.time()
# 제목

# 전처리 
pp = ns.PreprocessingText()
content_no_stopwords_title = pp.make_content_re(df_raw['title']) # 일반적인 불용어, 특수문자 제거

# 유효 명사 추가
noun_dict_route = 'data/미래에셋_단어사전.txt'
with open(noun_dict_route,encoding = 'utf-8-sig') as f: noun_proper = f.read()
noun_list = noun_proper.split('\n')
pp.add_noun_dict(noun_list) 

# 검색어에 대한 불용어 추가
stopword_dict_route = 'data/미래에셋_불용어사전.txt'
with open(stopword_dict_route,encoding = 'utf-8-sig') as f2: stopword = f2.read()
stopword_list = stopword.split('\n')
pp.add_stopwords(stopword_list) 

# 명사 추출
content_tk_title = pp.tokenize(content_no_stopwords_title)

toc = time.time()
print(f'excution time : {toc-tic}')

추가한 명사
['정보통신', '환헤지', '조웅기', '핀테크', '큐레이션', '투명성', '최경주', '상장', '임직원', '아시아그로쓰', '라이징스타', '파이낸셜', '캡스', '리파이낸싱', '싱가포르투자청', '도이체방크', '도이치뱅크', '카우룽', '그린마크', '광주은행', '대신증권', '삼성생명', '신한은행', '우리은행', '유안타증권', '펀드슈퍼마켓', '한국투자증권', '기업은행', '하나은행', '제일은행', '퀀트모델', '포트폴리오', '헤지펀드', '트레이딩', '영업점', '셀렉션', '구인', '사업화', '협약식', '미래에셋센터원', '최연소', '이테크', '미래에셋', '미래에셋대우', '에트리홀딩스', '펀드설정액', '분기', '벤처', '사회적기업', '성장지원', '헬스케어', '해결책', '메리츠', '신고가', '지주', '증권주', '벨류에이션', '이베스트', '수익성', '실사구시', '페어트레이딩', '쓸어', '상장사', '한투', '한국투자증권', '순매수', '거래량', '상장주식수', '분기', '미상환', '업무지구', '장기적', '한온시스템', '보여', '주간사', '젠바디', '예탁결제원', '크라우드펀딩', '영업이익률', '이익률', '파워리퍼블릭', '운용사', '아모텍', '지분', '박스권', '저평가', '시총', '긍정적', '납세자', '셀리드', '에스퓨얼셀', '게임사', '할인율', '재도전', '인공지능', '당기순익', '비대면', '사업가치', '재평가', '순매도', '순매수', '수혜주', '마중물', '혼조세', '신고가', '합리적', '재취업', '다각화', '자문형랩', '순익', '설정액', '원소재', '메디톡스', '인허가', '성장통', '호실적', '주간사', '주관사', '예상치', '호실적', '압도', '신성장', '기술료', '조기상환', '비임원', '기대감', '순매도', '저평가', '거래량', '내수', '건설사', 

In [7]:
# 전처리된 데이터 저장
df_raw['tokenized_doc'] = content_tk
df_raw['tokenized_title'] = content_tk_title
with open('tokenized data/tkdata_미래에셋대우.pickle','wb') as handle:
    pickle.dump(df_raw,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
df_raw

,date,title,content,category,date_tmp,tokenized_doc,tokenized_title
0,2019-04-19,"NH투자證 `선택과 집중` 통했다…펀드설정액, 증권사 1위에 올라",지난해 말 조직개편을 통해 펀드 판매에 승부수를 띄운 NH투자증권이 올해 들어 금융...,"[경제>증권_증시, 경제>금융_재테크, 경제>유통]",737168,"[조직개편, 펀드, 판매, 승, 부수, 투자, 증권, 금융, 권, 가장, 판매, 증...","[투자, 선택, 집중, 통, 펀드설정액, 증권사]"
1,2019-04-19,"2차전지 기업들, 속속 '코스닥 입성' 채비","코윈테크, 8월 코스닥 상장 추진 \n \n \n[ 이우상 기자 ] 2차전지 생산용...","[경제>산업_기업, 경제>증권_증시, 경제>금융_재테크]",737168,"[코윈, 테크, 코스닥, 상장, 추진, 차, 전지, 생산, 용, 설비, 제조, 기업...","[차, 전지, 기업, 코스닥, 입성, 채비]"
2,2019-04-19,증시 호조에 증권株 `훨훨`…2분기 실적도 `맑음`,올해 들어 증시가 호조를 보이면서 증권주들이 1분기 양호한 성적표를 받아들었다. 이...,"[경제>증권_증시, 경제>금융_재테크, 경제>국제경제]",737168,"[증시, 호조, 보이, 증권주, 양호, 성적표, 아들, 었다, 증권, 실적, 호조,...","[증시, 호조, 증권, 훨훨, 실적, 맑음]"
3,2019-04-19,"한화 LifePlus TDF 설정 1년, 4% 대 운용성과 달성",한화 라이프플러스 상장지수펀드(ETF)의 수익률이 설정 이후 양호한 수익률을 올리고...,"[경제>금융_재테크, 경제>증권_증시, 경제>국제경제]",737168,"[한화, 라이프, 플러스, 장지수, 펀드, 수익률, 설정, 양호, 수익률, 한화, ...","[한화, 설정, 운용, 성과, 달성]"
4,2019-04-19,"미래에셋벤처투자의 사회적기업1호펀드, 72.4% 수익 올리고 청산돼",미래에셋벤처투자는 지난 2011년 42억원으로 결성했던 사회적기업1호펀드가 72.4...,"[경제>산업_기업, 경제>증권_증시, 경제>국제경제]",737168,"[벤처, 투자, 결성, 사회적기업, 호, 펀드, 수익, 뒤, 지난달, 청산, 사회적...","[벤처, 투자, 사회적기업, 호, 펀드, 수익, 청산]"
5,2019-04-19,"미래에셋대우-에트리홀딩스, 중소·벤처기업 성장지원 업무 협약",[ 이송렬 기자 ] \n \n미래에셋대우는 미래에셋센터원에서 에트리홀딩스(주)와 '...,[경제>산업_기업],737168,"[미래에셋대우, 미래에셋센터원, 에트리홀딩스, 중소, 벤처기업, 성장지원, 업무, ...","[미래에셋대우, 에트리홀딩스, 중소, 벤처기업, 성장지원, 업무, 협약]"
6,2019-04-19,"미래에셋대우, 에트리홀딩스와 `중소·벤처기업 성장 지원` MOU 체결",미래에셋대우는 지난 18일 서울 중구 미래에셋센터원에서 에트리홀딩스와 '중소·벤처 ...,"[경제>산업_기업, 경제>취업_창업]",737168,"[미래에셋대우, 서울, 중구, 미래에셋센터원, 에트리홀딩스, 중소, 벤처, 기업, ...","[미래에셋대우, 에트리홀딩스, 중소, 벤처기업, 성장, 지원, 체결]"
7,2019-04-19,"고민 깊어지는 헬스케어, 해결책 찾기에 고심",[머니투데이 강상구 머니투데이방송 PD] \n\n\n미국종목에 대한 월가의 투자의견...,[],737168,"[머니투데이, 강상, 구, 머니투데이, 방송, 미국, 종목, 월, 투자, 의견, 국...","[고민, 헬스케어, 해결책, 찾기, 고심]"
8,2019-04-19,"[ET투자뉴스]GS_거래비중 기관 39.68%, 외국인 39.95%",업종평균에 비해서 변동폭 대비 수익률은 다소 부정적 \n \n19일 오전 10시5분...,"[경제>증권_증시, 경제>금융_재테크, 경제>국제경제]",737168,"[업종, 평균, 변동, 수익률, 부정, 오전, 하락, 보합, 권, 월간, 하락, 기...","[투자, 뉴스, 거래, 기관, 외국인]"
9,2019-04-19,"[ET투자뉴스]NAVER, ""1Q19, 예상보다…"" BUY(유지)-메리츠종금증권","메리츠종금증권에서 19일 NAVER(035420)에 대해 ""1Q19, 예상보다 견조...",[경제>국제경제],737168,"[메리츠, 종금, 증권, 견조할, 실적, 투자, 의견, 목표, 주가, 메리츠, 종금...","[투자, 뉴스, 메리츠, 종금, 증권]"
